In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
from collections import defaultdict
import csv
import numpy as np
from numpy import genfromtxt
import tabulate
from IPython.display import HTML
import pickle
from IPython.display import display
!pip install tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 34.1 MB/s eta 0:00:00


# Load Dataset

In [ ]:
item_train = genfromtxt('/content/drive/Shareddrives/BANGKIT 2023 CAPSTONE/CAPSTONE/DATASET/experimental_Seller_Features - Sheet1.csv', delimiter=',')
user_train = genfromtxt('/content/drive/Shareddrives/BANGKIT 2023 CAPSTONE/CAPSTONE/DATASET/experimental_User_Features - Sheet2.csv', delimiter=',')
y_train = genfromtxt('/content/drive/Shareddrives/BANGKIT 2023 CAPSTONE/CAPSTONE/DATASET/y_train - Sheet2.csv', delimiter=',')
with open('/content/drive/Shareddrives/BANGKIT 2023 CAPSTONE/CAPSTONE/DATASET/item_train header.txt', newline='') as f: #csv reader handles quoted strings better
  item_features = list(csv.reader(f))[0]
with open('/content/drive/Shareddrives/BANGKIT 2023 CAPSTONE/CAPSTONE/DATASET/user_train header.txt', newline='') as f:
  user_features = list(csv.reader(f))[0]

#item_vecs = genfromtxt('/content/drive/MyDrive/CAPSTONE/DATASET/Item Vecs - Sheet1.csv', delimiter=',')
seller_vecs = genfromtxt('/content/drive/Shareddrives/BANGKIT 2023 CAPSTONE/CAPSTONE/DATASET/experimental_Item Vecs - Sheet1.csv', delimiter=',')

menu_dict = defaultdict(dict)
count = 0
#    with open('./data/movies.csv', newline='') as csvfile:
with open('/content/drive/Shareddrives/BANGKIT 2023 CAPSTONE/CAPSTONE/DATASET/menu_dict - Sheet1.csv', newline='') as csvfile:
  reader = csv.reader(csvfile, delimiter=',', quotechar='"')
  for line in reader:
      if count == 0:
          count += 1  #skip header
          #print(line) print
      else:
          count += 1
          seller_id = int(line[0])
          menu_dict[seller_id]["nama_penjual"] = line[1]
          menu_dict[seller_id]["menu"] = line[2]

In [ ]:
num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 2  # remove seller id and 'none' column at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 2  # start of columns to use in training, items
print(f"Number of training vectors: {len(item_train)}")

Number of training vectors: 1000


# Visualizing Data

In [ ]:
# Create HTML tables for visualization
def create_html_table(data, header, max_rows=5):
    html_table = "<table>\n<thead>\n<tr>"
    html_table += "".join([f"<th>{h}</th>" for h in header])
    html_table += "</tr>\n</thead>\n<tbody>\n"
    for row in data[:max_rows]:
        html_table += "<tr>"
        html_table += "".join([f"<td>{col}</td>" for col in row])
        html_table += "</tr>\n"
    html_table += "</tbody>\n</table>"
    return html_table

In [ ]:
item_table_html = create_html_table(item_train, item_features, max_rows=5)
HTML(item_table_html)

seller_id,none,ave_rating,nasi_goreng,mie_ayam,tahu_tek,batagor,pentol,kacang_hijau,es_dawet,es_oyen,sate,siomay,es_campur,soto,seblak,rujak
1.0,2000.0,4.6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2.0,2000.0,2.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3.0,2000.0,3.8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,2000.0,4.8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.0,2000.0,2.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
user_table_html = create_html_table(user_train, user_features, max_rows=5)
HTML(user_table_html)

user_id,rating_count,rating_ave,nasi_goreng,mie_ayam,tahu_tek,batagor,pentol,kacang_hijau,es_dawet,es_oyen,sate,siomay,es_campur,soto,seblak,rujak
1.0,8.0,1.5,4.5,2.0,0.0,0.0,4.0,4.0,3.5,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2.0,15.0,3.185714286,3.4,1.0,0.0,5.0,1.2,4.8,4.8,4.8,2.0,2.0,3.4,3.8,4.8,3.6
3.0,20.0,2.7,3.8,1.8,3.8,4.0,3.4,1.2,2.4,4.2,2.2,0.2,1.8,4.4,0.2,4.4
4.0,16.0,3.057142857,2.8,1.8,4.8,3.6,4.4,3.4,2.8,4.6,3.4,3.2,1.2,2.6,2.6,1.6
5.0,20.0,2.671428571,4.0,1.4,2.6,3.6,2.4,2.6,2.0,5.0,1.4,1.2,1.0,0.8,4.6,4.8


# Preprocessing

**Scaling**

In [ ]:
# scale training data
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


**Train-Test Splitting**

In [ ]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"seller/item training data shape: {item_train.shape}")
print(f"seller/item test data shape: {item_test.shape}")

seller/item training data shape: (800, 17)
seller/item test data shape: (200, 17)


# Machine Learning Model

**Define Model Architecture**

In [ ]:
num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=num_outputs)
])

item_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=num_outputs)
])

In [ ]:
# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

In [ ]:
# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 14)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 15)]                 0         []                            
                                                                                                  
 sequential (Sequential)     (None, 32)                   40864     ['input_1[0][0]']             
                                                                                                  
 sequential_1 (Sequential)   (None, 32)                   41120     ['input_2[0][0]']             
                                                                                              

In [ ]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

**Training**

In [ ]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

Epoch 1/30
25/25 [==============================] - 3s 11ms/step - loss: 0.1375
Epoch 2/30
25/25 [==============================] - 0s 9ms/step - loss: 0.1036
Epoch 3/30
25/25 [==============================] - 0s 7ms/step - loss: 0.0865
Epoch 4/30
25/25 [==============================] - 0s 10ms/step - loss: 0.0688
Epoch 5/30
25/25 [==============================] - 0s 10ms/step - loss: 0.0625
Epoch 6/30
25/25 [==============================] - 0s 12ms/step - loss: 0.0557
Epoch 7/30
25/25 [==============================] - 0s 13ms/step - loss: 0.0387
Epoch 8/30
25/25 [==============================] - 0s 8ms/step - loss: 0.0330
Epoch 9/30
25/25 [==============================] - 0s 11ms/step - loss: 0.0241
Epoch 10/30
25/25 [==============================] - 0s 9ms/step - loss: 0.0169
Epoch 11/30
25/25 [==============================] - 0s 9ms/step - loss: 0.0158
Epoch 12/30
25/25 [==============================] - 0s 15ms/step - loss: 0.0115
Epoch 13/30
25/25 [=======================

In [ ]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

7/7 [==============================] - 0s 4ms/step - loss: 0.0880


0.08803819864988327

# Prediction

**Prediction for New User**

In [ ]:
def gen_user_vecs(user_vec, num_items):
    """ given a user vector return:
        user predict maxtrix to match the size of item_vecs """
    user_vecs = np.tile(user_vec, (num_items, 1))
    return user_vecs

def print_pred_movies(y_p, item, menu_dict, maxcount):
    """ print results of prediction of a new user. inputs are expected to be in
        sorted order, unscaled. """
    count = 0
    disp = [["y_p", "seller_id", "rating_ave", "nama_penjual", "menu"]]

    for i in range(0, y_p.shape[0]):
        if count == maxcount:
            break
        count += 1
        seller_id = item[i, 0].astype(int)
        disp.append([np.around(y_p[i, 0], 1), item[i, 0].astype(int), np.around(item[i, 2].astype(float), 1),
                     menu_dict[seller_id]['nama_penjual'], menu_dict[seller_id]['menu']])

    table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
    return table

In [ ]:
new_user_id = 1001 #user_id
new_rating_count = 30 #berapa kali menilai
new_rating_ave = 3.75 #rata-rata dari seluruh menu
nasi_goreng = 3.00 #rata-rata dari seluruh jenis makanan yang sama atau skor preferensi
mie_ayam = 3.00
tahu_tek = 3.00
batagor = 3.00
pentol = 4.00
kacang_hijau = 5.00
es_dawet = 3.00
es_oyen = 3.00
sate = 3.00
siomay = 3.00
es_campur = 3.00
soto = 3.00
seblak = 3.00
rujak = 3.00

user_vec = np.array([[new_user_id,new_rating_count,new_rating_ave,nasi_goreng,mie_ayam,tahu_tek,batagor,pentol,kacang_hijau,es_dawet,es_oyen,sate,siomay,es_campur,soto,seblak,rujak]])

In [ ]:
# generate and replicate the user vector to match the number movies in the data set.
user_vecs = gen_user_vecs(user_vec,len(seller_vecs))

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sseller_vecs = scalerItem.transform(seller_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sseller_vecs[:, i_s:]])

# unscale y prediction
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = seller_vecs[sorted_index]  #using unscaled vectors for display

HTML(print_pred_movies(sorted_ypu, sorted_items, menu_dict, maxcount = 10))

12/12 [==============================] - 0s 2ms/step


y_p,seller_id,rating_ave,nama_penjual,menu
4.6,125,5,es_oyen Bu V N,es_oyen
4.6,78,5,es_oyen Bu X J,es_oyen
4.6,166,4.6,es_oyen Bu H F,es_oyen
4.6,346,4.6,es_oyen Bu G P,es_oyen
4.5,320,4.4,es_oyen Pak R J,es_oyen
4.5,287,4.4,es_oyen Bu B H,es_oyen
4.5,266,4.4,es_oyen Bu G N,es_oyen
4.5,262,5,es_campur Bu O T,es_campur
4.5,69,5,kacang_hijau Pak V T,kacang_hijau
4.5,270,5,kacang_hijau Bu T C,kacang_hijau


In [ ]:
model.save("path/to/save/model")

In [ ]:
!tensorflowjs_converter --input_format=tf_saved_model \
                       --output_node_names='dot_1' \
                       path/to/saved_model \
                       path/to/tfjs_model

2023-12-16 13:42:54.447593: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-16 13:42:54.447687: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-16 13:42:54.451489: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-16 13:42:56.448513: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
"""
model_json = model.to_json()
with open("path/to/drive/model.json", "w") as json_file:
    json_file.write(model_json)
"""

In [ ]:
#from tensorflow.keras.models import load_model

In [ ]:
"""
from flask import Flask, request, jsonify

app = Flask(__name__)
model = load_model("path/to/save/model")

@app.route("/predict", methods=["POST"])

def predict(input_data):
    data = request.json
    # Perform inference using the loaded model
    #seller_vecs = genfromtxt('/content/drive/MyDrive/CAPSTONE/DATASET/experimental_Item Vecs - Sheet1.csv', delimiter=',')
    user_unscaled = data

    #scalerItem = StandardScaler()
    #scalerItem.fit(seller_vecs)
    #seller_vecs_scaled = scalerItem.transform(seller_vecs)

    scalerUser = StandardScaler()
    scalerUser.fit(user_unscaled)
    user_vecs_scaled = scalerUser.transform(user_unscaled)

    result = model.predict([user_vecs_scaled[:, 3:], seller_vecs_scaled[:, 2:]])
    return jsonify({"prediction": result.tolist()})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)
"""

SyntaxError: ignored